### Import packages

In [62]:
import os
import json
import string
import pandas as pd
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import PlaintextCorpusReader, stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

### Create documents corpus

In [ ]:
corpus_dir = "./Literature-original"
corpus = PlaintextCorpusReader(corpus_dir, ".*\.txt")
files_names = corpus.fileids()
files_names

### Corpus documents proprocessing

In [ ]:
documents = {}
for file_name in files_names:
    documents[file_name] = corpus.raw(file_name)
print(json.dumps(documents, indent=4, ensure_ascii=False))

In [ ]:
lengths = {}
for file_name in documents:
    lengths[file_name] = {
        "pre": len(word_tokenize(documents[file_name]))
    }
print(json.dumps(lengths, indent=4, ensure_ascii=False))

In [ ]:
ps = PorterStemmer()

In [ ]:
for file_name in documents:
    documents[file_name] = documents[file_name].lower()
    documents[file_name] = "".join([char for char in documents[file_name] if char not in string.punctuation])
    documents[file_name] = "".join([char for char in documents[file_name] if not char.isdigit()])
    documents[file_name] = " ".join([ps.stem(word) for word in word_tokenize(documents[file_name])])
    documents[file_name] = " ".join([word for word in word_tokenize(documents[file_name]) if word not in list(stopwords.words('english'))])

print(json.dumps(documents, indent=4, ensure_ascii=False))

In [ ]:
for file_name in documents:
    lengths[file_name]['post'] = len(word_tokenize(documents[file_name]))
print(json.dumps(lengths, indent=4, ensure_ascii=False))

In [ ]:
lengths = pd.DataFrame.from_dict(lengths, orient='index')

In [ ]:
lengths['diff'] = lengths['pre']-lengths['post']
lengths['pct'] = lengths['diff']/lengths['pre']
lengths

### Create frequency matrix

In [ ]:
docs = pd.DataFrame.from_dict(documents, orient="index")
docs.columns = ['content']
docs

In [57]:
cv = CountVectorizer()
matrix_tf = cv.fit_transform(docs['content'])
matrix_tf

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 6019 stored elements and shape (23, 2503)>

In [59]:
sparsity_tf = 1-(matrix_tf.getnnz()/(matrix_tf.shape[0]*matrix_tf.shape[1]))
sparsity_tf

0.8954472024874498

In [60]:
tv = TfidfVectorizer()
matrix_tfidf = tv.fit_transform(docs['content'])
matrix_tfidf

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6019 stored elements and shape (23, 2503)>

In [61]:
sparsity_tfidf = 1-(matrix_tfidf.getnnz()/(matrix_tfidf.shape[0]*matrix_tfidf.shape[1]))
sparsity_tfidf

0.8954472024874498

### Directories for results

In [63]:
if not os.path.exists("./wodclouds"):
    os.mkdir("./wodclouds")
if not os.path.exists("./topic_modelling"):
    os.mkdir("./topic_modelling")
if not os.path.exists("./topic_modelling/topics"):
    os.mkdir("./topic_modelling/topics")
if not os.path.exists("./topic_modelling/documents"):
    os.mkdir("./topic_modelling/documents")
if not os.path.exists("./clustering"):
    os.mkdir("./clustering")
if not os.path.exists("./ngrams"):
    os.mkdir("./ngrams")

### Worldclouds

### Topic modeling

### Clustering

### N-grams